In [1]:
#Packages
import math
from pathlib import Path 
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt

from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPooling2D 
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

In [2]:
with open("C:\\Users\\Uriel\\Desktop\\TP Thèse Doctoral ML\\1- CNN\\shipsnet.json") as data_file:
    dataset = json.load(data_file) # charger les données
ships = pd.DataFrame(dataset)
ships.head()

,data,labels,locations,scene_ids
0,"[82, 89, 91, 87, 89, 87, 86, 86, 86, 86, 84, 8...",1,"[-118.2254694333423, 33.73803725920789]",20180708_180909_0f47
1,"[76, 75, 67, 62, 68, 72, 73, 73, 68, 69, 69, 6...",1,"[-122.33222866289329, 37.7491755586813]",20170705_180816_103e
2,"[125, 127, 129, 130, 126, 125, 129, 133, 132, ...",1,"[-118.14283073363218, 33.736016066914175]",20180712_211331_0f06
3,"[102, 99, 113, 106, 96, 102, 105, 105, 103, 10...",1,"[-122.34784341495181, 37.76648707436548]",20170609_180756_103a
4,"[78, 76, 74, 78, 79, 79, 79, 82, 86, 85, 83, 8...",1,"[-122.34852408322172, 37.75878462398653]",20170515_180653_1007


In [3]:
# On va extraire "data et labels" pour la suite de notre étude
ships = ships[["data","labels"]]
ships.head()

,data,labels
0,"[82, 89, 91, 87, 89, 87, 86, 86, 86, 86, 84, 8...",1
1,"[76, 75, 67, 62, 68, 72, 73, 73, 68, 69, 69, 6...",1
2,"[125, 127, 129, 130, 126, 125, 129, 133, 132, ...",1
3,"[102, 99, 113, 106, 96, 102, 105, 105, 103, 10...",1
4,"[78, 76, 74, 78, 79, 79, 79, 82, 86, 85, 83, 8...",1


In [4]:
# 0 : pour les images avec navire
# 1 : pour les images sans navire

img_ships = ships["labels"].value_counts()[0]
img_no_ships = ships["labels"].value_counts()[1]

# Afficher le nombre d'images 

print("Nombre d'image avec navire: {}".format(img_ships),"\n")
print("Nombre d'image sans navire: {}".format(img_no_ships))

Nombre d'image avec navire: 3000 

Nombre d'image sans navire: 1000


In [5]:
# On va transformer les informations json en tableau numpy 
# et ensuite les assigner comme variables x et y

import numpy as np
x = np.array(dataset["data"]).astype("uint8")
y = np.array(dataset["labels"]).astype("uint8")

x.shape

(4000, 19200)

In [6]:
x_reshaped = x.reshape([-1, 3, 80, 80]).transpose([0, 2, 3, 1])
x_reshaped.shape

(4000, 80, 80, 3)

In [7]:
from tensorflow.keras.utils import to_categorical
y.shape
y_reshaped = to_categorical(y, num_classes = 2)
y_reshaped.shape

(4000, 2)

In [8]:
x_reshaped = x_reshaped / 255

# Normaliser les valeurs RVB du premier pixel de la première image de l'ensemble de données.
x_reshaped[0][0][0]

array([0.32156863, 0.36862745, 0.31372549])

In [9]:
# Maintenant les données sont prêtes à être divisées en données d'entrainement et de test.

x_train, x_test, y_train, y_test = train_test_split(x_reshaped, y_reshaped, test_size = 0.2, random_state = 42)
x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size = 0.20, random_state = 42)

print("x_train shape",x_train.shape)
print("x_test shape",x_test.shape)

print("y_train shape",y_train.shape)
print("y_test shape",y_test.shape)

print("y_train shape",x_validation.shape)
print("y_train shape",y_validation.shape)

x_train shape (2560, 80, 80, 3)
x_test shape (800, 80, 80, 3)
y_train shape (2560, 2)
y_test shape (800, 2)
y_train shape (640, 80, 80, 3)
y_train shape (640, 2)


In [10]:
""" ResNet101 """
from keras.applications.resnet import ResNet101
from keras.preprocessing import image
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import SGD

base_model = ResNet101(weights='imagenet',include_top=False, input_shape=(80, 80, 3))

x = base_model.output
x = Flatten()(x)

# Ajouter la nouvelle couche fully-connected pour la classification à 2 classes
predictions = Dense(2, activation='softmax')(x)

# Définir le nouveau modèle
new_model = Model(inputs=base_model.input, outputs=predictions)
#model = Model(input=base_model.input, output=base_model.get_layer('block4_pool').output)

new_model.compile(loss="categorical_crossentropy", optimizer=SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])
new_model.summary()
# Entraîner sur les données d'entraînement (X_train, y_train)
#history = new_model.fit(x_train, y_train, epochs=50, batch_size=32,validation_data=(x_test, y_test))
# model_info = new_model.fit(data, target, epochs=30, batch_size=32)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 80, 80, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 86, 86, 3)    0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 40, 40, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 40, 40, 64)   256         ['conv1_conv[0][0]']             
                                                                                              

C:\Users\Uriel\miniconda3\envs\ML\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


                                                                                                  
 conv2_block3_1_conv (Conv2D)   (None, 20, 20, 64)   16448       ['conv2_block2_out[0][0]']       
                                                                                                  
 conv2_block3_1_bn (BatchNormal  (None, 20, 20, 64)  256         ['conv2_block3_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_1_relu (Activatio  (None, 20, 20, 64)  0           ['conv2_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block3_2_conv (Conv2D)   (None, 20, 20, 64)   36928       ['conv2_block3_1_relu[0][0]']    
          

                                                                                                  
 conv3_block3_1_conv (Conv2D)   (None, 10, 10, 128)  65664       ['conv3_block2_out[0][0]']       
                                                                                                  
 conv3_block3_1_bn (BatchNormal  (None, 10, 10, 128)  512        ['conv3_block3_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block3_1_relu (Activatio  (None, 10, 10, 128)  0          ['conv3_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block3_2_conv (Conv2D)   (None, 10, 10, 128)  147584      ['conv3_block3_1_relu[0][0]']    
          

                                                                                                  
 conv4_block2_1_conv (Conv2D)   (None, 5, 5, 256)    262400      ['conv4_block1_out[0][0]']       
                                                                                                  
 conv4_block2_1_bn (BatchNormal  (None, 5, 5, 256)   1024        ['conv4_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block2_1_relu (Activatio  (None, 5, 5, 256)   0           ['conv4_block2_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block2_2_conv (Conv2D)   (None, 5, 5, 256)    590080      ['conv4_block2_1_relu[0][0]']    
          

...

 on)                                                                                              
                                                                                                  
 conv4_block11_3_conv (Conv2D)  (None, 5, 5, 1024)   263168      ['conv4_block11_2_relu[0][0]']   
                                                                                                  
 conv4_block11_3_bn (BatchNorma  (None, 5, 5, 1024)  4096        ['conv4_block11_3_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 conv4_block11_add (Add)        (None, 5, 5, 1024)   0           ['conv4_block10_out[0][0]',      
                                                                  'conv4_block11_3_bn[0][0]']     
                                                                                                  
 conv4_blo

 lization)                                                                                        
                                                                                                  
 conv4_block14_add (Add)        (None, 5, 5, 1024)   0           ['conv4_block13_out[0][0]',      
                                                                  'conv4_block14_3_bn[0][0]']     
                                                                                                  
 conv4_block14_out (Activation)  (None, 5, 5, 1024)  0           ['conv4_block14_add[0][0]']      
                                                                                                  
 conv4_block15_1_conv (Conv2D)  (None, 5, 5, 256)    262400      ['conv4_block14_out[0][0]']      
                                                                                                  
 conv4_block15_1_bn (BatchNorma  (None, 5, 5, 256)   1024        ['conv4_block15_1_conv[0][0]']   
 lization)

 conv4_block17_out (Activation)  (None, 5, 5, 1024)  0           ['conv4_block17_add[0][0]']      
                                                                                                  
 conv4_block18_1_conv (Conv2D)  (None, 5, 5, 256)    262400      ['conv4_block17_out[0][0]']      
                                                                                                  
 conv4_block18_1_bn (BatchNorma  (None, 5, 5, 256)   1024        ['conv4_block18_1_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 conv4_block18_1_relu (Activati  (None, 5, 5, 256)   0           ['conv4_block18_1_bn[0][0]']     
 on)                                                                                              
                                                                                                  
 conv4_blo

 lization)                                                                                        
                                                                                                  
 conv4_block21_1_relu (Activati  (None, 5, 5, 256)   0           ['conv4_block21_1_bn[0][0]']     
 on)                                                                                              
                                                                                                  
 conv4_block21_2_conv (Conv2D)  (None, 5, 5, 256)    590080      ['conv4_block21_1_relu[0][0]']   
                                                                                                  
 conv4_block21_2_bn (BatchNorma  (None, 5, 5, 256)   1024        ['conv4_block21_2_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 conv4_blo

 conv5_block1_2_conv (Conv2D)   (None, 3, 3, 512)    2359808     ['conv5_block1_1_relu[0][0]']    
                                                                                                  
 conv5_block1_2_bn (BatchNormal  (None, 3, 3, 512)   2048        ['conv5_block1_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block1_2_relu (Activatio  (None, 3, 3, 512)   0           ['conv5_block1_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv5_block1_0_conv (Conv2D)   (None, 3, 3, 2048)   2099200     ['conv4_block23_out[0][0]']      
                                                                                                  
 conv5_blo

In [ ]:
# Entraîner sur les données d'entraînement (X_train, y_train)
history = new_model.fit(x_train, y_train, epochs=50, batch_size=32,validation_data=(x_test, y_test))
#new_model.summary()

Epoch 1/50
15/80 [====>.........................] - ETA: 6:50 - loss: 0.7691 - accuracy: 0.6854

In [ ]:
new_model.evaluate(x_test, y_test)

In [ ]:
# Courbe de perte
plt.figure(figsize=[8,6])
plt.plot(history.history['loss'],'green',linewidth=3.0)
plt.plot(history.history['val_loss'],'orange',linewidth=3.0)
plt.legend(['Training_loss', 'Validation_loss)'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Courbe de perte(loss)',fontsize=16)

# Courbe de precision(Accuracy)
plt.figure(figsize=[8,6])
plt.plot(history.history['accuracy'],'green',linewidth=3.0)
plt.plot(history.history['val_accuracy'],'orange',linewidth=3.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Accuracy',fontsize=16)
plt.title('Courbe de precision(Accuracy)',fontsize=16)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn import metrics
import seaborn as sns

Y_pred = new_model.predict(x_test)
# Convertir les classes de prédictions en un vecteur chaud.
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
# Convertissez les observations de validation en un vecteur chaud.
Y_true = np.argmax(y_test,axis = 1)

# Calculer la matrice de confusion

print("\n""Test Accuracy Score : ",metrics.accuracy_score(Y_true, Y_pred_classes),"\n")

confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 

# Plot the confusion matrix
f,ax = plt.subplots(figsize=(8, 6))
sns.heatmap(confusion_mtx, annot=True, linewidths=0.01,linecolor="gray", fmt= '.1f',ax=ax)
plt.xlabel("Predictions Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.tight_layout()

plt.show()

In [ ]:
prediction = new_model.predict(x_test)
pd.Series(prediction[1], index=["Image sans navire", "Image avec navire"])

In [ ]:
plt.imshow(x_test[1])
plt.axis("off")
plt.show()